# Mark: 86/100

### Notes: 

Good solutions to many tasks. Be careful of casting things to ints (task 3). A small issue in tasks 5/6 gave slightly incorrect result. Good attempt at bonus task.

# Checkpoint 3

**Due: Friday, 4 December, 2020 at 5:00pm GMT**

### Read This First
1. Use the constants provided in the cell below. Do not use your own constants.

2. Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc. **Additional cells need to be in the proper order such that your code runs correctly the first time through.**

3. **IMPORTANT!** Before submitting your notebook, clear the output by clicking *Restart & Clear Output* from the *Kernel* menu. If you do not do this, the file size of your notebook will be very large.


# NMR spectrum of water

This problem concerns obtaining the nuclear magnetic resonance (NMR) spectrum of water from a raw NMR signal.

Given: a file containing the free-induction decay signal $y(t)$ (electromagnetic radiation emitted by protons in water), following an RF $\pi$-pulse of f=60MHz. The recorded signal is frequency-subtracted, i.e., the radio-frequency output signal has been mixed with the carrier frequency f=60MHz to shift the signal to lower frequencies in the range of hundreds of Hz.

The signal has three main components: the actual NMR signal from protons in water molecules, white noise, and a 50 Hz "mains hum" (electrical signal picked up by the sensitive NMR detector, coming from the mains AC; the signal also contains higher harmonics).

### Hints on how to solve this checkpoint:
- The code must work (and will be tested on) for other data files, in which the frequency of the water peak may differ by 10%. Any 'fine-tuning' of your algorithms so that they only work for the given data file is therefore discouraged.
- Write the code in a modular way so that you can re-use functions from previous tasks. This will save you a lot of time.
- Do not make the code more complex that it needs to be. Classes, complex data structures etc. are not required for this checkpoint.
- Use NumPy/SciPy functions rather than your own implementation whenever possible.
- As in CP1 and CP2, apart from numerical accuracy, efficiency and coding style will also be marked. Try to make your code readable.
- Comment on the results obtained. This may help to get a better mark if there is a problem with the code.

**There are 6 tasks in this CP worth 100 points, plus a bonus task 7 worth 15 points. The total mark will be the sum of all marks, or 100 points, whichever is lower.**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas
import time
from scipy import integrate, optimize

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 16

# Task 1 (15p)

Load the data from the file "signals/water_16_samples.csv" and make a plot of the recorded signal versus time for t=[0,0.05), for the first of the 16 samples. Label the axes.

Each row of the data file (except the first which contains table headings) has the following format:

time, s1, s2, s3, ...

where "time" is in seconds, and s1,s2,s3,... represent the NMR signal (arbitrary units) from independent realisations of the experiment. Differences between the samples should be only due to noise; it is the same experiment repeated 16  times.

In [ ]:
#signals = pandas.read_csv(filepath_or_buffer='file:///home/jovyan/PHYS100902020-1SV1SEM1/Checkpoint 3/signals/water_16_samples.csv')

signals = pandas.read_csv(filepath_or_buffer='signals/water_16_samples.csv')

In [ ]:
# times array will be used in most subsequent functions
times = signals['t']
signal1 = signals['s1']

print(times.size)
# Times is an array of 30000 places long and it contains data from t=0 to t=0.5
# Data up to the point t=0.05 is then stored up to the index of 0.05/0.5 * 30000 = 3000

plt.plot(times[:3000], signal1[:3000])
plt.xlabel('Time [s]')
plt.ylabel('Signal Strength')
plt.title('Signal number 1')
plt.show()

In [ ]:
print('Number of points is ' + str(times.size))
print('Total time sampled is ' + str(times.max()) + ' s')
print('Sampling frequency is ' + str(times.size/times.max()) + ' Hz')
print('Nyquist frequency of this sample is ' + str(times.size/(2*times.max())) + ' Hz')
print('This means that the frequencies that are going to be computed range from -15000 Hz to 15000 Hz')
print('Index in the frequency spectrum the represents a change of ' + str(times.size/times.max() / times.size) + ' Hz which is the frequency space resolution')

# Task 1: 12/15

Solution is correct in this case, but the hard-coded array indexing would have failed for samples with different sampling or timing.

# Task 2 (15p)

Calculate the amplitude spectrum of the NMR signal, and plot it as a function of frequency f [Hertz], for f=0 to 500 Hz. Assume the length of the time series is tmax=0.5s.

**Note: in workshop, you have been plotting wavenumber ($k = \frac{2\pi}{\lambda}$), whereas frequency is $f = \frac{1}{\lambda}$.**

In [ ]:
tmax = 0.5
def plot_amplitude_spectrum(FFTsignal, timevalues, maxfreq, signalNumber):
    """
    Function takes in the fourier transformed values of a signal and plots its (normalized?) amplitude spectrum up to maxfreq
    signal -> array of data points
    timevalues -> array of times when the signal was sampled
    maxfreq -> maximum frequency up to which the spectrum is plotted
    signalNumber -> string that is the nubmer of the signal
    """
    
    nmbPoints = FFTsignal.size
    freq = np.linspace(1, (nmbPoints - 1) / tmax, nmbPoints) # frequency domain
    
    # Plot from 0 to maxfreq Hz
    nn = int(maxfreq * tmax) # maxfreq /(1/tmax) = maxfreq * tmax
    
    # Add some multiplying factors to the plot to normalize it
    plt.plot(freq[:nn], 2 * abs(FFTsignal[:nn]) / timevalues.size)
    plt.title('Amplitude spectrum of signal ' + signalNumber)
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('$\\bar{f} (f)$')
    plt.show()

In [ ]:
# Find the fourier transform
FFTsignal1 = np.fft.rfft(signal1)
plot_amplitude_spectrum(FFTsignal1, timevalues=times, maxfreq=500, signalNumber='1')

There are clearly two main signals, one at 50 Hz which is just the 'hum' from the AC supply, and the second one that is centered around 300 Hz, which must be the actual NMR signal!

# Task 2: 12/15

Plot looks nice. Implementation uses a slightly awkward combination of function arguments and global data that have to be supplied just so to work correctly.

# Task 3 (15p)

Filter out the noise by passing the signal through a bandpass filter centered at the water peak with a width $\pm$30 Hz. Plot the filtered signal y(t) for t=[0,0.5).

In [ ]:
def filter_signal_around_peak(signal, timevalues, filterWidth):
    """
    Takes a signal, computes its discrete fourier transform, finds the frequency of the strongest signal,
    filters the signal around the peak and finally returns the inverse transform of the filtered amplitude signal.
    signal -> array of signal values
    timevalues -> array of times at which the signal was sampled
    filterWidth -> scalar that tells how wide the filter (in units of Hz) is to be (total width is taken to be 2 * filterWidth)
    """
    
    # Transform the signal
    FFTsignal = np.fft.rfft(signal)
    
    # Index of the heighest peak in the transformed data
    indexMax = np.argmax(abs(FFTsignal))

    # One index is equal to a change of this much Hz
    deltaIndex = int(1 / tmax)

    # filterWidth Hz is equal to this much index change
    deltaFilterHz = int(filterWidth / deltaIndex)

    # Filter the amplitude spectrum of the signal
    nmbPoints = FFTsignal.size
    freq = np.linspace(0, (nmbPoints-1) / tmax, nmbPoints) # frequency domain
    FFTsignal[np.abs(freq - freq[indexMax]) > filterWidth] = 0
    
    # Find the inverse transform which is the filtered signal
    filteredSignal = np.fft.irfft(FFTsignal)
    
    return filteredSignal, FFTsignal

In [ ]:
filteredSignal1, filteredFFTsignal1 = filter_signal_around_peak(signal1, timevalues=times, filterWidth=30)

plot_amplitude_spectrum(filteredFFTsignal1, timevalues=times, maxfreq=500, signalNumber='1')

# Plot the filtered signal
plt.plot(times, filteredSignal1)
plt.xlabel('Time [s]')
plt.ylabel('Strength of the signal')
plt.title('Filtered signal 1')
plt.xlim(0, 0.05) # inserted by grader
plt.show()

# Task 3: 13/15

The answer is correct, but `deltaIndex = int(1 / tmax)` only works because `1 / tmax` is an integer. A different tmax would have resulted in round-off error (`deltaIndex` would be 0 for `tmax > 1`).

# Task 4 (15p)

Use non-linear curve fitting to fit the function:

$
\Large
\begin{align}
y_{\rm theor}(t) = A \sin(2\pi f_0 t) e^{-t/t_0}
\end{align}
$

to the filtered signal for t=[0,tmax), with unknown parameters $A, f_0, t_0$. Find and print out the best-fit frequency $f_0$. The frequency should be accurate to within 0.01 Hz of the correct answer.

In [ ]:
def theoretical_function(t, A, f0, t0):
    """
    A sin(2*pi*f0*t)*exp(-t/t0)
    
    Parameters are: t, A, f0, t0
    """
    return A * np.sin(2*np.pi*f0*t) * np.exp(-t/t0)

In [ ]:
nmbPoints = filteredFFTsignal1.size
freq = np.linspace(0, (nmbPoints-1) / tmax, nmbPoints) # frequency domain

# Find the frequency with the biggest amplitude, meaning the actual value is very near this frequency
# This ensures the code works for both datasets, and hopefully for other ones as well
biggestFrequency = freq[np.argmax(abs(filteredFFTsignal1))]

popt, pcov = optimize.curve_fit(theoretical_function, times, filteredSignal1, p0=[2, biggestFrequency, 1])
fity = theoretical_function(times, *popt)
print(popt)
print(f'Best fit frequency is {popt[1]} Hz')

plt.plot(times, filteredSignal1, label='Filtered signal', color='blue')
plt.plot(times, fity, label='Best fit to the filtered signal', color='red')
plt.xlabel('Time [s]')
plt.ylabel('Strength of the signal')
plt.title('Filtered signal and best fit to the filtered signal')
plt.legend()
plt.xlim(0, 0.05)
plt.show()

In [ ]:
print ("There will be tests here. Great job so far!")
### BEGIN HIDDEN TESTS
print ("Correct answer is 302.40 Hz.")
### END HIDDEN TESTS

# Task 4: 20/20

Great!

# Task 5 (15p)

Determine $f_0$ in a different way: find the position of the water peak in the amplitude spectrum by fitting the curve

$
\large
\begin{align}
\tilde{y}_{\rm theor}(f) = C + \frac{A}{\sqrt{\lambda^4 + (f^2 - f_0^2)^2 + 2 \lambda^2(f^2 + f_0^2))}}
\end{align}
$

This curve comes from Fourier-transforming the exponentially damped sine function from the previous task, plus a constant C to account for background noise. $\lambda$ denotes the damping rate (inversely proportional to $t_0$ from task 4).

As before, use only the first of the 16 samples for this task. The frequency should be accurate to within 0.01 Hz of the correct answer.

In [ ]:
def amplitude_theoretical_function(f, C, A, lam, f0):
    """
    C + A / sqrt(lam^4 + (f^2 - f0^2)^2 + 2*lam^2*(f^2 + f0^2))
    Parameters are: f, C, A, lam, f0
    """
    return C + A/(np.sqrt(lam**4 + (f**2 - f0**2)**2 + 2*lam**2*(f**2 + f0**2)))

In [ ]:
parameters, covariance = optimize.curve_fit(amplitude_theoretical_function, freq, abs(filteredFFTsignal1), p0=[153, 8000, 0.5, 307.02])
print(f'Best fit frequency is {parameters[3]} Hz')

fity = amplitude_theoretical_function(freq, *parameters)

plt.plot(freq[:250], abs(filteredFFTsignal1)[:250], label='Filtered signal')
plt.plot(freq[:250], fity[:250], label='Best fit to the filtered signal')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Strength of the signal')
plt.title('Signal and its best fit in the frequency domain')
plt.legend()
plt.show()

In [ ]:
print ("Tests here. Keep it up!")
### BEGIN HIDDEN TESTS
print ("Correct answer is 302.40 Hz.")
### END HIDDEN TESTS

# Task 5: 15/20

Answer is slightly incorrect due to fitting the entire filtered signal that includes the zeroed out portion. Otherwise, well done.

# Task 6 (10p)

Determine the frequency $f_0$ for all 16 data sets using the method from Task 5. Calculate mean $f_0$ and its standard error. The frequency should be accurate to within 0.01 Hz of the correct answer and the standard error should be within 1%.

In [ ]:
frequencies = []
for i in range(16):
    signal = signals['s' + str(i+1)] # extract the signal
    
    # Filter the signal
    filteredSignal, filteredFFTsignal = filter_signal_around_peak(signal, timevalues=times, filterWidth=30)
    
    # Find the best fit frequency for this signal in the frequency space
    parameters, covariance = optimize.curve_fit(amplitude_theoretical_function, freq, abs(filteredFFTsignal), p0=[153, 8000, 0.5, 307.02])
    
    # Save the best fit frequency
    frequencies.append(parameters[3])

# Since the function we are fitting to the data is even, the frequency that gets calculated can be both positive and negative
# Since it doesn't matter whether it is positive or negative, I just take the absolute value making all the frequencies positive
frequencies = np.abs(frequencies)
meanFreq = np.mean(frequencies)
standardError = np.std(frequencies)/np.sqrt(signals.shape[1]-1) #-1 to get rid of the inital collumn where collumn names are defined

print(f'Mean value of the frequency is {meanFreq} Hz and its standard error is {standardError} Hz')

In [ ]:
print ("Tests here. Almost there!")
### BEGIN HIDDEN TESTS
print ("Correct answer is 302.37861566541795 +/- 0.03724493085391285 Hz.")
### END HIDDEN TESTS

# Task 6: 10/15

Incorrect answer for reason stated above. Otherwise, a nice, neat solution.

# Bonus: Task 7 (15p)

Find the 95% equally-tailed credible interval of $f_0$ from task 5 using Bayesian inference.

Use only the first of the 16 samples for this task. Assume the spectrum can be modelled by the curve from task 5, with $C=0$ and random noise superimposed on the curve. The noise should be generated as independent, identically distributed random numbers $\{\chi_k\}$ drawn from the Chi distribution with two degrees of freedom and unknown amplitude $\sigma$, so that the amplitude spectrum is

$
\large
\begin{align}
\tilde{y}_k = \frac{A}{\sqrt{\lambda^4 + (f_k^2 - f_0^2)^2 + 2\lambda^2(f_k^2 + f_0^2))}} + \sigma \chi_k
\end{align}
$

where $f_k = k/t_{max}$. The rationale for using the Chi distribution comes from Fourier-transforming Gaussian noise and taking its modulus (to plot the amplitude spectrum).

Does the average value of $f_0$ obtained in task 6 lie in the credible interval? The bounds of the interval should be accurate to within 0.01 Hz of the correct answer.

In [ ]:
from scipy import stats

In [ ]:
# Chi distribution with two degrees of freedom random number
print(stats.chi.rvs(df=2))

# Previously calculated values, we expect the newly calculated values to be near these values
# This affects our choice of priors
print(f'Previously calculated parameters were C={parameters[0]}, A={parameters[1]}, lam={parameters[2]}, f0={parameters[3]}')

In [ ]:
def simulate_data(f, A, lam, f0, sigma):
    """
    Function takes in parameters A, lam, f0, sigma and simulates data based
    on those parameters and random noise drawn from the chi distribution with
    two degrees of freedom.
    Data simulated is A / sqrt(lam^4 + (f^2 - f0^2)^2 + 2*lam^2*(f^2 + f0^2)) + sigma*chi
    """
    
    errors = np.sqrt(np.array([stats.chi.rvs(df=2) for i in range(len(freq))]))
    simulatedData = A / np.sqrt(lam**4 + (f**2 - f0**2)**2 + 2*lam**2*(f**2 + f0**2)) + sigma * errors
    return simulatedData

def squared_distance(actualData, simulatedData):
    """
    Function takes in real data and simulated data and calculates the total sum of the residuals.
    This is taken to be a distance function in the Approximate Bayesian Computation.
    Parameters: actualData, simulatedData
    """
    distance = np.sum((actualData - simulatedData)**2)
    return distance

def ABC(nmbSim, eps, data, ranges, model_func, dist_func):
    """
    Approximate Bayesian Computation. 
    Parameters are 
    nmbSim -> number of simulations
    eps -> threshold for the distance function, above which values are rejected
    data -> actual data
    ranges -> a list of lists of floats, a list of min, max values for each model parameter
    model_func -> model function
    dist_func -> Distance function
    
    Values of parameters are drawn from uniform distributions centered around previously
    calculated values.
    """
    
    # Lists to store successful parameters
    selected = []
    
    for i in range(nmbSim):
        pars = [np.random.uniform(*r) for r in ranges]
        #A = np.random.uniform(low=6133344, high=6133346)
        #lam = np.random.uniform(low=-0.5, high=-0.4)
        #f0 = np.random.uniform(low=306, high=310)
        #sigma = np.random.uniform(low=-2, high=-1)
        
        simulatedData = model_func(freq, *pars)
        distance = dist_func(data, simulatedData)

        if distance < eps:
            print('Accepted!!', end=' ')
            selected.append(pars)
    
    return np.array(selected)

In [ ]:
# mean of chi with two degrees of freedom is sqrt(pi/2)
# Our calculated C value is -1.81845, to get a similar value sigma must be around -1.45091
# Use these values to estimate the epsilon to be used in subsequent calculations
# I tested different values of epsilon and the one I'm using right now is the one that accepts around 1% of simulated data
data = simulate_data(f=freq, A=6123281.43, lam=0.444419, f0=307.0167, sigma=-1.809099)
nn = int(500 * times.max()) # maxfreq /(1/timevalues.max()) = maxfreq * timevalues.max()
plt.plot(freq[:nn], abs(filteredFFTsignal1[:nn]), label='Actual Data')
plt.plot(freq[:nn], data[:nn], label='Simulated Data')
plt.legend()
plt.xlabel('Frequency [Hz]')
plt.ylabel('$\\bar{f} (f)$')
plt.show()

firstDistance = squared_distance(abs(filteredFFTsignal1), data)
print(f'First distance is {firstDistance}')

# Pull values of parameters from uniform distribution centered around previously calculated values
As = [6133344, 6133346]
lams = [-0.5, -0.4]
f0s = [freq[np.argmax(abs(filteredFFTsignal1))] - 2, freq[np.argmax(abs(filteredFFTsignal1))] + 2]
sigmas = [-5, -1]
ranges = [As, lams, f0s, sigmas]

t1 = time.time()
paras = ABC(nmbSim=500, eps=8000000, data=abs(filteredFFTsignal1), ranges=ranges, 
          model_func=simulate_data, dist_func=squared_distance)
t2 = time.time()
print(f'Time taken is {t2 - t1} seconds')

frequencies = paras[:,2]
# Find the mean and the standard deviation of successful f0 parameters
meanf0 = np.mean(frequencies)
stdf0 = np.std(frequencies)
print(f'Mean value of the frequency is {meanf0} Hz and its standard deviation is {stdf0} Hz')

In [ ]:
frequencies = np.sort(frequencies)
# 2.5% = 1 / 40, so take the parameters at indices N/40 and N-N/40.
print(f"(f0-, f0+) = {frequencies[len(frequencies)//40]}, {frequencies[-len(frequencies)//40]}.")

Frequency calculated in task 6 does fall within the 95% credible interval

In [ ]:
print ("Tests. Nice work!")
### BEGIN HIDDEN TESTS
print ("Correct answer is 302.37760310745193 to 302.4166345063264 Hz.")
### END HIDDEN TESTS

# Task 7: 4/15

Answer from task 6 does not fall within your 95% credible interval. Approach looks good, but answers are systemmatically too high and credible interval is about 4x too wide. I think there was also an issue in the `simulate_data` function adding too much noise (should be one value of sigma per run, not one per frequency bin). Still, a good attempt, deserves some points.